In [1]:
import numpy as np
import pandas as pd

Data can be found here: https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data

In [2]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_test_id = df_test["id"]

oil = pd.read_csv("oil.csv")
oil['dcoilwtico'] = oil['dcoilwtico'].bfill()

he = pd.read_csv("holidays_events.csv")
stores = pd.read_csv("stores.csv")

In [3]:
df = pd.merge(df, oil, on='date', how='left')
df_test = pd.merge(df_test, oil, on='date', how='left')

df = pd.merge(df, stores, on='store_nbr', how='left')
df_test = pd.merge(df_test, stores, on='store_nbr', how='left')

# Feature Reduction

In [4]:
holidays = he.drop(he[ (he.transferred) | (he.type == 'Work Day') ].index)
holidays = holidays.drop(holidays[holidays.date.duplicated()].index)
holidays = holidays.drop(['type', 'description', 'transferred'], axis=1)

In [5]:
holidays

,date,locale,locale_name
0,2012-03-02,Local,Manta
1,2012-04-01,Regional,Cotopaxi
2,2012-04-12,Local,Cuenca
3,2012-04-14,Local,Libertad
4,2012-04-21,Local,Riobamba
...,...,...,...
344,2017-12-22,Local,Salinas
346,2017-12-23,National,Ecuador
347,2017-12-24,National,Ecuador
348,2017-12-25,National,Ecuador


In [6]:
national = holidays[holidays.locale == 'National']
regional = holidays[holidays.locale == 'Regional']
local = holidays[holidays.locale == 'Local']

pd.options.mode.chained_assignment = None
national = national.drop(['locale', 'locale_name'], axis=1)
national['holidays'] = 1

df = pd.merge(df, national, on='date', how='left')
df_test = pd.merge(df_test, national, on='date', how='left')

In [7]:
for i, j in zip(regional.date, regional.locale_name):
    df['holidays'][(df.date == i) & (df.state == j)] = 1
    df_test['holidays'][(df_test.date == i) & (df_test.state == j)] = 1

In [8]:
for i, j in zip(local.date, local.locale_name):
    df['holidays'][(df.date == i) & (df.city == j)] = 1
    df_test['holidays'][(df_test.date == i) & (df_test.city == j)] = 1

In [9]:
df

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,type,cluster,holidays
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,93.14,Quito,Pichincha,D,13,1.0
1,1,2013-01-01,1,BABY CARE,0.000,0,93.14,Quito,Pichincha,D,13,1.0
2,2,2013-01-01,1,BEAUTY,0.000,0,93.14,Quito,Pichincha,D,13,1.0
3,3,2013-01-01,1,BEVERAGES,0.000,0,93.14,Quito,Pichincha,D,13,1.0
4,4,2013-01-01,1,BOOKS,0.000,0,93.14,Quito,Pichincha,D,13,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,47.57,Quito,Pichincha,B,6,NaN
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,47.57,Quito,Pichincha,B,6,NaN
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,47.57,Quito,Pichincha,B,6,NaN
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.57,Quito,Pichincha,B,6,NaN


In [10]:
df_test

,id,date,store_nbr,family,onpromotion,dcoilwtico,city,state,type,cluster,holidays
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.80,Quito,Pichincha,D,13,NaN
1,3000889,2017-08-16,1,BABY CARE,0,46.80,Quito,Pichincha,D,13,NaN
2,3000890,2017-08-16,1,BEAUTY,2,46.80,Quito,Pichincha,D,13,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,46.80,Quito,Pichincha,D,13,NaN
4,3000892,2017-08-16,1,BOOKS,0,46.80,Quito,Pichincha,D,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,47.26,Quito,Pichincha,B,6,NaN
28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26,Quito,Pichincha,B,6,NaN
28509,3029397,2017-08-31,9,PRODUCE,1,47.26,Quito,Pichincha,B,6,NaN
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,Quito,Pichincha,B,6,NaN


# LabelEncoder

In [11]:
from sklearn.preprocessing import LabelEncoder
headers = ['family', 'city', 'state', 'type']

df['dcoilwtico'] = df['dcoilwtico'].bfill()
df_test['dcoilwtico'] = df_test['dcoilwtico'].bfill()

for i in headers:
    le = LabelEncoder()
    le_test = LabelEncoder()
    df[i] = le.fit_transform(df[i])
    df_test[i] = le_test.fit_transform(df_test[i])
    
df['holidays'] = df['holidays'].replace(to_replace=np.nan, value=0)
df_test['holidays'] = df_test['holidays'].replace(to_replace=np.nan, value=0)

In [12]:
train_target = df['sales']
train_features = df.drop(['sales', 'id', 'date'], axis=1)
test_features = df_test.drop(['id', 'date'], axis=1)

In [13]:
train_features

,store_nbr,family,onpromotion,dcoilwtico,city,state,type,cluster,holidays
0,1,0,0,93.14,18,12,3,13,1.0
1,1,1,0,93.14,18,12,3,13,1.0
2,1,2,0,93.14,18,12,3,13,1.0
3,1,3,0,93.14,18,12,3,13,1.0
4,1,4,0,93.14,18,12,3,13,1.0
...,...,...,...,...,...,...,...,...,...
3000883,9,28,0,47.57,18,12,1,6,0.0
3000884,9,29,1,47.57,18,12,1,6,0.0
3000885,9,30,148,47.57,18,12,1,6,0.0
3000886,9,31,8,47.57,18,12,1,6,0.0


# Principal Component Analysis (PCA)

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

In [15]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95, svd_solver='full')
reduced_train = pca.fit_transform(train_features_scaled)
reduced_test = pca.transform(test_features_scaled)

In [16]:
pca.explained_variance_

array([0.15022329, 0.12953074, 0.09312123, 0.08854227, 0.07587585,
       0.07159983, 0.03886317])

# Training Models with All Features

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error as msle
state = 0
ests = 10
depth = 32
samples = 0.7
x_train, x_test, y_train, y_test = train_test_split(reduced_train, train_target, test_size=0.1, random_state=state)

In [18]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(random_state = state, max_depth = depth)
dtr.fit(x_train, y_train)

print( msle( y_train, dtr.predict(x_train), squared=False ) )
print( msle( y_test, dtr.predict(x_test), squared=False ) )
print(dtr.feature_importances_)

0.48501712578291534
0.9200026935305629
[0.07892119 0.17245244 0.07246246 0.55288767 0.05654662 0.01930687
 0.04742274]


In [19]:
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor(n_estimators = ests, random_state = state, max_depth = depth, bootstrap=True, max_samples = samples)
et.fit(x_train, y_train)

print( msle( y_train, et.predict(x_train), squared=False ) )
print( msle( y_test, et.predict(x_test), squared=False ) )
print(et.feature_importances_)

0.7266223071145427
0.9243269420924688
[0.08200478 0.11570831 0.04907001 0.63255907 0.04672402 0.02327064
 0.05066317]


In [20]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = ests, random_state = state, max_depth = depth, bootstrap=True, max_samples = samples)
rf.fit(x_train, y_train)

print( msle( y_train, rf.predict(x_train), squared=False ) )
print( msle( y_test, rf.predict(x_test), squared=False ) )
print(rf.feature_importances_)

0.7066204559431989
0.9011208332128668
[0.08167187 0.18332305 0.06552975 0.54821508 0.04963775 0.01961611
 0.05200638]


# Feature Reduction Based On Importance

In [21]:
train_features2 = df.drop(['sales', 'id', 'date', 'city', 'state', 'type', 'holidays'], axis=1)
test_features2 = df_test.drop(['id', 'date', 'city', 'state', 'type', 'holidays'], axis=1)

In [22]:
train_features2

,store_nbr,family,onpromotion,dcoilwtico,cluster
0,1,0,0,93.14,13
1,1,1,0,93.14,13
2,1,2,0,93.14,13
3,1,3,0,93.14,13
4,1,4,0,93.14,13
...,...,...,...,...,...
3000883,9,28,0,47.57,6
3000884,9,29,1,47.57,6
3000885,9,30,148,47.57,6
3000886,9,31,8,47.57,6


In [23]:
scaler2 = MinMaxScaler()
train_features_scaled2 = scaler2.fit_transform(train_features2)
test_features_scaled2 = scaler2.transform(test_features2)

In [24]:
pca2 = PCA(n_components=0.95, svd_solver='full')
reduced_train2 = pca2.fit_transform(train_features_scaled2)
reduced_test2 = pca2.transform(test_features_scaled2)

In [25]:
pca2.explained_variance_

array([0.0926928 , 0.09063219, 0.08854227, 0.08029912])

In [26]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(reduced_train2, train_target, test_size=0.1, random_state = state)

# Training with Reduced Features

In [27]:
dtr2 = DecisionTreeRegressor(random_state = state, max_depth = depth)
dtr2.fit(x_train2, y_train2)

print( msle( y_train2, dtr2.predict(x_train2), squared=False ) )
print( msle( y_test2, dtr2.predict(x_test2), squared=False ) )
print(dtr2.feature_importances_)

0.48885359761519354
0.8675937460273074
[0.10958319 0.13281599 0.56320098 0.19439985]


In [28]:
et2 = ExtraTreesRegressor(n_estimators = ests, random_state = state, max_depth = depth, bootstrap=True, max_samples = samples)
et2.fit(x_train2, y_train2)

print( msle( y_train2, et2.predict(x_train2), squared=False ) )
print( msle( y_test2, et2.predict(x_test2), squared=False ) )
print(et2.feature_importances_)

0.7153239178574702
0.9057352928470691
[0.06644659 0.12706829 0.63501699 0.17146813]


In [29]:
rf2 = RandomForestRegressor(n_estimators = ests, random_state = state, max_depth = depth, bootstrap=True, max_samples = samples)
rf2.fit(x_train2, y_train2)

print( msle( y_train2, rf2.predict(x_train2), squared=False ) )
print( msle( y_test2, rf2.predict(x_test2), squared=False ) )
print(rf2.feature_importances_)

0.6624040801274704
0.8429068585930589
[0.10359859 0.13774581 0.56310678 0.19554882]


# Model Selection

RandomForestRegressor with reduced features was selected as it had the best performance (lowest RMSLE)

In [30]:
def create_file(model, name, test_set=reduced_test2):
    predictions = pd.DataFrame(np.round(model.predict(test_set), 5))
    predictions = predictions.rename({0: "sales"}, axis=1)
    pd.concat([df_test_id, predictions],axis = 1).to_csv(name+".csv",index = False)

In [31]:
ests = 20
rf_submit1 = RandomForestRegressor(n_estimators = ests, max_depth = depth, bootstrap=True, max_samples = samples)
rf_submit2 = RandomForestRegressor(n_estimators = ests, max_depth = depth, bootstrap=True, max_samples = samples)
rf_submit3 = RandomForestRegressor(n_estimators = ests, max_depth = depth, bootstrap=True, max_samples = samples)
rf_submit4 = RandomForestRegressor(n_estimators = ests, max_depth = depth, bootstrap=True, max_samples = samples)
rf_submit5 = RandomForestRegressor(n_estimators = ests, max_depth = depth, bootstrap=True, max_samples = samples)

models = [rf_submit1, rf_submit2, rf_submit3, rf_submit4, rf_submit5]

for model in models:
    model.fit(reduced_train2, train_target)
    print( msle( train_target, model.predict(reduced_train2), squared=False ) )

0.6678215157190076
0.6687960908172379
0.6683526143490469
0.6683064546867218
0.6679591980820861


In [32]:
num = 1
for model in models:
    name = "submission"+str(num)
    num+=1
    create_file(model, name)